# K-média

## 1. Preparações do ambiente

In [1]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt, pow

%matplotlib inline


## 2. K-médias

### 2.1 Pseudo código elaborado

``` python 
def Kmedias(dataset, k, num_iters):
    centroides = centroides_aleatorios(teste, k)
    lista_centroides = []
    for i in range(num_iters):
        // Definindo qual centroid mais próximo
        for j in range(numero_de_objetos):
            centroide = centroide_mais_proximo(objeto, centroides)
            lista_centroides.append(centroide)
        
        centroides = recalcular_centroides()
    
    teste["centroide"] = lista_centroides
    teste.drop(axis='columns', labels=['d1', 'd2'])
```



### 2.2 Algoritmo

In [5]:
def centroides_aleatorios(dataset, k):
    atrib1_max = dataset["d1"].max()
    atrib1_min = dataset["d1"].min()
    
    atrib2_max = dataset["d2"].max()
    atrib2_min = dataset["d2"].min()
    
    clusters = []
    for i in range(k):
        a1 = random.uniform(atrib1_min, atrib1_max)
        a2 = random.uniform(atrib2_min, atrib2_max)
        
        clusters.append([a1, a2])

    return clusters

In [6]:
#Distancia Euclidiana
def distEuclidiana(p, centroide):
    distancia = sqrt(pow((p[0] - centroide[0]), 2) + pow((p[1] - centroide[1]), 2))
    return distancia

In [7]:
def centroides_mais_proximos(d1, d2, centroides, numero_de_objetos):
    lista_centroides = []
    for i in range(numero_de_objetos):
        centroide = centroide_mais_proximo([d1[i], d2[i]], centroides)
        lista_centroides.append(centroide)
    
    return lista_centroides

In [8]:
def centroide_mais_proximo(objeto, centroides):
    menor_dist = distEuclidiana(objeto, centroides[0])
    index = 0
    for i in range(1, len(centroides)):
        nova_dist = distEuclidiana(objeto, centroides[i])
        if nova_dist < menor_dist:
            menor_dist = nova_dist
            index = i
    
    return index

In [14]:
def recalcular_centroides(dataset, k):
    novos_centroides = []
    for i in range(k):
        qtd_obj = len(dataset.query(f'clusters == {i}'))
        d1 = dataset.query(f'clusters == {i}')['d1'].sum()
        d2 = dataset.query(f'clusters == {i}')['d2'].sum()
        
        d1 /= qtd_obj
        d2 /= qtd_obj
        
        novos_centroides.append([d1, d2])
    
    return novos_centroides

In [15]:
def salvar_arquivo(nome, dataset):
    dataset.drop(axis='columns', labels=['d1', 'd2']).to_csv(f'datasets/{nome}.csv')

In [23]:
def kmedias(dataset, k, numero_de_iteracoes, nome):
    centroides = centroides_aleatorios(dataset, k)
    lista_centroides = []
    numero_de_objetos = len(dataset)
    
    for i in range(numero_de_iteracoes):
        lista_centroides = centroides_mais_proximos(dataset["d1"], dataset["d2"], centroides, numero_de_objetos)
        
        dataset['clusters'] = lista_centroides
        centroides = recalcular_centroides(dataset, k)
    
    
    salvar_arquivo(nome, dataset)
    return [dataset, centroides]